In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import regex as re
from sklearn.feature_extraction.text import TfidfVectorizer
import wordcloud
from wordcloud import WordCloud
import nltk
from nltk.stem import WordNetLemmatizer 
nltk.download('wordnet')
import spacy
nlp = spacy.load('en_core_web_sm')
nltk.download('stopwords')

<h1> Preprocessing W.R.T Bert </h1>


In [ ]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

In order for torch to use the GPU, we need to identify and specify the GPU as the device. Later, in our training loop, we will load data onto the device. 

In [ ]:
import torch
import torch.nn as nn
# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

Installing the Hugging Face library (transformers)

## Loading the data

In [ ]:
train_inputs = pd.read_csv('../input/bert-hate-speech-train-test-validation/train_inputs.csv').values.astype(np.int_)
train_masks = pd.read_csv('../input/bert-hate-speech-train-test-validation/train_masks.csv').values.astype(np.int_)
train_labels = pd.read_csv('../input/bert-hate-speech-train-test-validation/train_labels.csv').values.astype(np.int_)

validation_inputs = pd.read_csv('../input/bert-hate-speech-train-test-validation/validation_inputs.csv').values.astype(np.int_)
validation_masks = pd.read_csv('../input/bert-hate-speech-train-test-validation/validation_masks.csv').values.astype(np.int_)
validation_labels = pd.read_csv('../input/bert-hate-speech-train-test-validation/validation_labels.csv').values.astype(np.int_)

test_inputs = pd.read_csv('../input/bert-hate-speech-train-test-validation/test_inputs.csv').values.astype(np.int_)
test_masks = pd.read_csv('../input/bert-hate-speech-train-test-validation/test_masks.csv').values.astype(np.int_)
test_labels = pd.read_csv('../input/bert-hate-speech-train-test-validation/test_labels.csv').values.astype(np.int_)

<h2> Converting inputs to bert embedding </h2>


In [ ]:
# Convert all inputs and labels into torch tensors, the required datatype 
# for our model.
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
test_inputs = torch.tensor(test_inputs)

train_labels = torch.tensor(train_labels.reshape((-1)).tolist())
validation_labels = torch.tensor(validation_labels.reshape((-1)).tolist())
test_labels = torch.tensor(test_labels.reshape((-1)).tolist())


train_masks = torch.tensor(train_masks)     
validation_masks = torch.tensor(validation_masks)
test_masks = torch.tensor(test_masks)

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here.
# For fine-tuning BERT on a specific task, the authors recommend a batch size of
# 16 or 32.

batch_size = 32

# Create the DataLoader for our training set.
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# Create the DataLoader for our validation set.
validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

# Create the DataLoader for our validation set.
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=validation_sampler, batch_size=batch_size)

In [ ]:
from transformers import AutoModel, BertTokenizerFast
bert = AutoModel.from_pretrained('bert-base-uncased')

In [ ]:
# freeze all the parameters
for param in bert.parameters():
    param.requires_grad = False

In [ ]:
class BERT_Arch(nn.Module):

    def __init__(self, bert):

        super(BERT_Arch, self).__init__()

        self.bert = bert 

        # dropout layer
        self.dropout = nn.Dropout(0.1)

        # relu activation function
        self.relu =  nn.ReLU()

        # dense layer 1
        self.fc1 = nn.Linear(768,512)

        # dense layer 2 (Output layer)
        self.fc2 = nn.Linear(512,2)

        #softmax activation function
        self.softmax = nn.LogSoftmax(dim=1)

    #define the forward pass
    def forward(self, sent_id, mask):

        #pass the inputs to the model  
        _, cls_hs = self.bert(sent_id, attention_mask=mask)

        
#         print(cls_hs.shape)
        
        x = self.fc1(cls_hs)

        x = self.relu(x)

        x = self.dropout(x)

        # output layer
        x = self.fc2(x)

        # apply softmax activation
        x = self.softmax(x)

        return x

In [ ]:
# pass the pre-trained BERT to our define architecture
model = BERT_Arch(bert)

# push the model to GPU
model = model.to(device)

In [ ]:
# optimizer from hugging face transformers
from transformers import AdamW

# define the optimizer
optimizer = AdamW(model.parameters(),
                  lr = 2e-5)          # learning rate

In [ ]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))


There is a class imbalance in our dataset. The majority of the observations are not spam. So, we will first compute class weights for the labels in the train set and then pass these weights to the loss function so that it takes care of the class imbalance.

In [ ]:
from sklearn.utils.class_weight import compute_class_weight

#compute the class weights
class_weights = compute_class_weight('balanced', np.unique(train_labels.cpu().detach().numpy()), train_labels.cpu().detach().numpy().flatten())

print("Class Weights:",class_weights)

In [ ]:
# converting list of class weights to a tensor
weights= torch.tensor(class_weights,dtype=torch.float)

# push to GPU
weights = weights.to(device)

# define the loss function
cross_entropy  = nn.NLLLoss(weight=weights) 

# number of training epochs
epochs = 300

In [ ]:
# function to train the model
def train():
  
    model.train()

    total_loss, total_accuracy = 0, 0
  
    # empty list to save model predictions
    total_preds=[]

    # iterate over batches
    for step,batch in enumerate(train_dataloader):

        # progress update after every 50 batches.
        if step % 50 == 0 and not step == 0:
            print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(train_dataloader)))

        # push the batch to gpu
        batch = [r.to(device) for r in batch]

        sent_id, mask, labels = batch

        # clear previously calculated gradients 
        model.zero_grad()        

        # get model predictions for the current batch
        preds = model(sent_id, mask)

        # compute the loss between actual and predicted values
        loss = cross_entropy(preds, labels)

        # add on to the total loss
        total_loss = total_loss + loss.item()

        # backward pass to calculate the gradients
        loss.backward()

        # clip the the gradients to 1.0. It helps in preventing the exploding gradient problem
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # update parameters
        optimizer.step()

        # model predictions are stored on GPU. So, push it to CPU
        preds=preds.detach().cpu().numpy()

        # append the model predictions
        total_preds.append(preds)

    # compute the training loss of the epoch
    avg_loss = total_loss / len(train_dataloader)

    # predictions are in the form of (no. of batches, size of batch, no. of classes).
    # reshape the predictions in form of (number of samples, no. of classes)
    total_preds  = np.concatenate(total_preds, axis=0)

    #returns the loss and predictions
    return avg_loss, total_preds


In [ ]:
# function for evaluating the model
def evaluate():

    print("\nEvaluating...")

    # deactivate dropout layers
    model.eval()

    total_loss, total_accuracy = 0, 0

    # empty list to save the model predictions
    total_preds = []

    # iterate over batches
    for step,batch in enumerate(validation_dataloader):

        # Progress update every 50 batches.
        if step % 50 == 0 and not step == 0:


            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(validation_dataloader)))

        # push the batch to gpu
        batch = [t.to(device) for t in batch]

        sent_id, mask, labels = batch

        # deactivate autograd
        with torch.no_grad():

            # model predictions
            preds = model(sent_id, mask)

            # compute the validation loss between actual and predicted values
            loss = cross_entropy(preds,labels)

            total_loss = total_loss + loss.item()

            preds = preds.detach().cpu().numpy()

            total_preds.append(preds)

    # compute the validation loss of the epoch
    avg_loss = total_loss / len(validation_dataloader) 

    # reshape the predictions in form of (number of samples, no. of classes)
    total_preds  = np.concatenate(total_preds, axis=0)
    print(total_preds)
    return avg_loss, total_preds

In [ ]:
from sklearn.metrics import f1_score
# set initial loss to infinite
best_valid_loss = float('inf')
best_f1_score = 0.0
# empty lists to store training and validation loss of each epoch
val_true = validation_labels.detach().cpu().numpy().reshape((-1,1)) # the actual predictions
# print(val_true.shape)
train_losses=[]
valid_losses=[]
f1_scores_validation = []
#for each epoch
for epoch in range(epochs):
    t0 = time.time()
    print('\n Epoch {:} / {:}'.format(epoch + 1, epochs))
    
    #train model
    train_loss, _ = train()
    
    #evaluate model
    valid_loss, val_preds = evaluate()
    val_preds =  np.argmax(val_preds,axis = 1).reshape((-1,1))
    f1_score_val = f1_score(val_true,val_preds)
    print(f1_score_val)
    
    #save the best model
    if f1_score_val > best_f1_score:
        best_f1_score = f1_score_val
        torch.save(model.state_dict(), 'saved_weights.pt')

    # append training and validation loss
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    f1_scores_validation.append(f1_score_val)
    elapsed = format_time(time.time() - t0)
    print('Elapsed Time', elapsed)   
    print(f'\nTraining Loss: {train_loss:.3f}')   
    print(f'Validation Loss: {valid_loss:.3f}')

In [ ]:
plt.plot(train_losses)   
plt.plot(valid_losses)            
plt.show()           

In [ ]:
plt.plot(f1_scores_validation) 

In [ ]:
#load weights of best model
path = 'saved_weights.pt'
model.load_state_dict(torch.load(path))

In [ ]:
# get predictions for validation data
val_preds = np.array([]).reshape((-1,2))
for step,batch in enumerate(validation_dataloader):

    # Progress update every 50 batches.
    if step % 50 == 0 and not step == 0:


        # Report progress.
        print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(validation_dataloader)))

    # push the batch to gpu
    batch = [t.to(device) for t in batch]

    sent_id, mask, labels = batch

    # deactivate autograd
    with torch.no_grad():

        # model predictions
        preds = model(sent_id, mask)

        preds = preds.detach().cpu().numpy()
        val_preds = np.concatenate((val_preds,preds))

In [ ]:
temp = val_preds

In [ ]:
from sklearn.metrics import classification_report
val_true = validation_labels.detach().cpu().numpy().reshape((-1,1))
val_preds =  np.argmax(val_preds,axis = 1).reshape((-1,1))
print(classification_report(val_true, val_preds))

## END

## Train Our Classification Model

from transformers import BertForSequenceClassification, AdamW, BertConfig

# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 2, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

# Tell pytorch to run this model on the GPU.
model.cuda()

Just for curiosity's sake, we can browse all of the model's parameters by name here.

In the below cell, We have printed out the names and dimensions of the weights for:

1. The embedding layer.
2. The first of the twelve transformers.
3. The output layer.




# Get all of the model's parameters as a list of tuples.
params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

## Optimizer & Learning Rate Scheduler
Now that we have our model loaded we need to grab the training hyperparameters from within the stored model.

For the purposes of fine-tuning, the authors recommend choosing from the following values:
- Batch size: 16, 32  (We chose 32 when creating our DataLoaders).
- Learning rate (Adam): 5e-5, 3e-5, 2e-5  (We'll use 2e-5).
- Number of epochs: 2, 3, 4  (We'll use 4).

The epsilon parameter `eps = 1e-8` is "a very small number to prevent any division by zero in the implementation" (from [here](https://machinelearningmastery.com/adam-optimization-algorithm-for-deep-learning/)).



# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )


from transformers import get_linear_schedule_with_warmup

# Number of training epochs (authors recommend between 2 and 4)
epochs = 10

# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

# Training Loop

### Function for calculating accuracy

import numpy as np
from sklearn.metrics import confusion_matrix,f1_score,precision_score,recall_score
 

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    print(type(pred_flat),type(labels_flat))
    f1 = f1_score(labels_flat,pred_flat)
    precision = precision_score(labels_flat,pred_flat)
    recall = recall_score(labels_flat,pred_flat)

    print(f1,precision,recall)
    return [np.sum(pred_flat == labels_flat) / len(labels_flat), f1, precision,recall]

### Function for calculation of elapsed time

import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))


import random

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128


# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# Store the average loss after each epoch so we can plot them.
loss_values = []
eval_f1 = []
eval_precision = []
eval_recall = []
# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # This will return the loss (rather than the model output) because we
        # have provided the `labels`.
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        outputs = model(b_input_ids, 
                    token_type_ids=None, 
                    attention_mask=b_input_mask, 
                    labels=b_labels)
        
        # The call to `model` always returns a tuple, so we need to pull the 
        # loss value out of the tuple.
        loss = outputs[0]

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over the training data.
    avg_train_loss = total_loss / len(train_dataloader)            
    
    # Store the loss value for plotting the learning curve.
    loss_values.append(avg_train_loss)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    eval_f1_temp,eval_precision_temp,eval_recall_temp, eval_accuracy = 0, 0, 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        
        # Telling the model not to compute or store gradients, saving memory and
        # speeding up validation
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # This will return the logits rather than the loss because we have
            # not provided labels.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # Get the "logits" output by the model. The "logits" are the output
        # values prior to applying an activation function like the softmax.
        logits = outputs[0]

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # Calculate the accuracy for this batch of test sentences.
        metrics = flat_accuracy(logits,label_ids)
        tmp_eval_accuracy = metrics[0]
        eval_f1_temp += metrics[1]
        eval_precision_temp += metrics[2]
        eval_recall_temp += metrics[3]
        
        # Accumulate the total accuracy.
        eval_accuracy += tmp_eval_accuracy
        
        # Track the number of batches
        nb_eval_steps += 1
    
    # Report the final accuracy for this validation run.
    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))
    eval_f1.append(eval_f1_temp/nb_eval_steps)
    eval_precision.append(eval_precision_temp/nb_eval_steps)
    eval_recall.append(eval_recall_temp/nb_eval_steps)

print("")
print("Training complete!")

Let's take a look at our training loss over all batches:

import matplotlib.pyplot as plt


import seaborn as sns

# Use plot styling from seaborn.
sns.set(style='darkgrid')

# Increase the plot size and font size.
sns.set(font_scale=1.5)
plt.rcParams["figure.figsize"] = (12,6)

# Plot the learning curve.
# plt.plot(loss_values, 'b-o')
plt.plot(eval_f1,'b-o')
# Label the plot.
plt.title("Training loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")

plt.show()

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = .2,random_state =0 )

print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)

#first we need to make the text document

# we will be creating bag of words model using tfidf
vectorizer = TfidfVectorizer(
    tokenizer = tokenize,
    ngram_range = (1,2),
    use_idf = True,
    max_features = 10000
    # min_df = ...
    # max_df = ...
)
vectorizer.fit(X_train)

tfIdf_train = vectorizer.transform(X_train)
tfIdf_test = vectorizer.transform(X_test)

print(vectorizer.vocabulary_,vectorizer.get_params,vectorizer)

tfIdf_train = tfIdf_train.toarray()
tfIdf_test = tfIdf_test.toarray()

vocab = {v:i for i, v in enumerate(vectorizer.get_feature_names())}
idf_vals = vectorizer.idf_
idf_dict = {i:idf_vals[i] for i in vocab.values()} #keys are indices; values are IDF scores

tfIdf_train.shape

#We will train an svm classifier
from sklearn.svm import SVC
classifier = SVC(kernel = 'linear')
classifier.fit(tfIdf_train,y_train)

y_pred = classifier.predict(tfIdf_test)

from sklearn.metrics import confusion_matrix,f1_score,precision_score,recall_score

cm = confusion_matrix()
f1 = f1_score(y_test,y_pred)
precision = precision_score(y_test,y_pred)
recall = recall_score(y_test,y_pred)

print(f1,precision,recall)
print(cm)

# # We will do random search 
# /*params = {
#     kernel:['linear', 'poly', 'rbf', 'sigmoid', 'precomputed'],
# }*/




# need to create a bag of words model
# consider bigrams ? lets go one by one
bow = vectorizer.transform(df.iloc[:,2])
y = df.iloc[:,1]

#splitting the data set 
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(bow,y,test_size = .2,random_state =0)

# we have got a bow model
# for starters maybe we can run  a logistic regression on it
from sklearn.linear_model import LogisticRegression
regressor = LogisticRegression()
regressor.fit(X_train,y_train)

y_pred = regressor.predict(X_test)

from sklearn.metrics import confusion_matrix,precision_score,recall_score,f1_score
cm = confusion_matrix(y_test,y_pred)
print(precision_score(y_test,y_pred),recall_score(y_test,y_pred),f1_score(y_test,y_pred))

#first we need to make the text document


# we will be creating bag of words model using tfidf
vectorizer = TfidfVectorizer(max_features=2000)
vectorizer.fit(df.iloc[:,2])

# need to create a bag of words model
# consider bigrams ? lets go one by one
bow = vectorizer.transform(df.iloc[:,2])
y = df.iloc[:,1]
#splitting the data set 
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(bow,y,test_size = .2,random_state =0)

# we have got a bow model
# for starters maybe we can run  a logistic regression on it
from sklearn.linear_model import LogisticRegression
regressor = LogisticRegression()
regressor.fit(X_train,y_train)

y_pred = regressor.predict(X_test)

from sklearn.metrics import confusion_matrix,precision_score,recall_score,f1_score
cm = confusion_matrix(y_test,y_pred)
print(precision_score(y_test,y_pred),recall_score(y_test,y_pred),f1_score(y_test,y_pred))

In [ ]:
#2000 - 0.5288135593220338
#3000 - 0.5017301038062284
#4000 - 0.48686514886164617